In [49]:
using Pkg
Pkg.add("LinearAlgebra")
Pkg.add("Plots")
Pkg.add("Random")
Pkg.add("Distances")
using LinearAlgebra
using Plots
using Random
using Distances

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
   Installed Distances ─ v0.10.7
    Updating `~/.julia/environments/v1.7/Project.toml`
  [b4f34e82] + Distances v0.10.7
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [b4f34e82] + Distances v0.10.7
Precompiling project...
  ✓ Distances
  1 dependency successfully precompiled in 2 seconds (141 already precompiled)


# Nice

## Neurons

In [3]:
function generate_memories(
        N,
        n
        )
    """
    Generate n random N neuron memories.
    Columns neuron states
    Rows memory
    """
    return bitrand((N,n))
end 

function initialise_weights(
        memories
        )
    """
    Initialise and return weight matrix.
    
    Given the matrix of memories to store where each column 
    is a state, each row a memory, return a matrix of
    weights between neurons in network via Hebbian 
    learning prescription.
    """
    # Initialise Tij weight matrix 
    N = length(memories[:,1])
    n = length(memories[1,:])
    T = zeros(
        Int8,
        (N,N)
    )
    
    # Iterate via Hebbian learning rule
    for ii in 1:N
        for jj in 1:N
            for kk in 1:n
                T[ii,jj] += (2*memories[ii,kk] - 1)*(2*memories[jj,kk]-1)
            end
        end
    end         
    return T
end

function initialise_network(
    N
)
"""
Initialise and return network of neurons.
"""
return bitrand((N,1))
end

initialise_network (generic function with 1 method)

## Learning

In [66]:
function learning_step(
        weights,
        neuron_states,
        activations
        )
    """
    Update the firing state of one neuron randomly via 
    Hopfield's learning prescription.
    
    Args
    ----
    weights: matrix
        matrix of weights between neurons 
    
    neuron_states: vector
        vector of current binary neuron states in the
        system. 
    
    activations: vector 
        The activation threshold for each neuron.
    
    Returns 
    -------
    updated_neuron_states: vector
    """
    # Select a neuron to update 
    selection = rand(1:length(neuron_states))
    
    # Compute the input to the neuron 
    input = dot(weights[selection,:], neuron_states)

    # Update the neuron state
    if input > activations[selection]
        neuron_states[selection] = 1
    else
        neuron_states[selection] = 0
    end

    # Return the updated neuron states
    return neuron_states
end

function learn(
        weights,
        memories,
        activations,
        n_steps
        )
    """
    Learn the network via Hebbian learning.
    
    Args
    ----
    weights: matrix
        matrix of weights between neurons 
    
    memories: matrix
        matrix of memories to store where each column 
        is a state, each row a memory.
    
    activations: vector 
        The activation threshold for each neuron.
    
    n_steps: int
        The number of learning steps to perform.
    
    Returns 
    -------
    neuron_states: matrix
        The final states of the neurons.
    """
    # Initialise the network
    neuron_states = initialise_network(length(memories[:,1]))
    
    # Iterate over the learning steps
    for ii in 1:n_steps
        neuron_states = learning_step(
            weights,
            neuron_states,
            activations
        )
    end
    
    # Return the final states of the neurons
    return neuron_states
end

function peturb_memory(
    memory,
    index
    )
    """
    Peturb a memory at a given index.

    Args
    ----
    memory: vector
        The memory to peturb.
    index: int
        The location of the neuron to peturb in the memory.
    """

    peturb = copy(memory)

    # If the neuron is 1, set it to 0, otherwise set it to 1
    if peturb[index] == 1
        peturb[index] = 0
    else
        peturb[index] = 1
    end
    
    return peturb
end

peturb_memory (generic function with 1 method)

# Rough

## Learning

In [5]:
# Run test
# Have 10 neurons and 5 memories
neurons = 10
memories = 10

# Generate random memories
memories = generate_memories(neurons,memories)

# Initialise weights
weights = initialise_weights(memories)

10×10 Matrix{Int8}:
 10   4   2   0   0   0   0   0  -4  -4
  4  10   0  -2   2  -2   2   2   2  -2
  2   0  10   4   4   0   0  -4  -4   0
  0  -2   4  10  -2  -2   6   2  -2  -2
  0   2   4  -2  10  -6  -2  -2  -2  -2
  0  -2   0  -2  -6  10  -2  -2   2   6
  0   2   0   6  -2  -2  10   2   2   2
  0   2  -4   2  -2  -2   2  10   2  -2
 -4   2  -4  -2  -2   2   2   2  10   2
 -4  -2   0  -2  -2   6   2  -2   2  10

In [6]:
network = initialise_network(neurons)

# Perform learning 
steps = 100

# Activations as array of length neurons all set to 0 
activations = zeros(Int8,(neurons,1))


10×1 Matrix{Int8}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [7]:
# Iterate through learning 
for ii in 1:steps
    network = learning_step(weights,network,activations)
    print('\n', network)
end


Bool[0; 1; 0; 1; 1; 0; 0; 0; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 0; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]
Bool[0; 1; 0; 1; 1; 0; 1; 1; 1; 0;;]


In [8]:
# Memory recall 
# See if the network is able to stabilise on a trained memory 
# Recall the first memory
recall = memories[:,1]

# Perform learning 
steps = 100

# Activations as array of length neurons all set to 0 
activations = zeros(Int8,(neurons,1))

# Perform learning
for ii in 1:steps
    update = learning_step(weights,recall,activations)
    # Calculate the distance between update and recall
    distance = norm(update-recall)
    # Print the distance
    print('\n', distance)
end


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

# Petrubed Memory

In [20]:
peturbed
peturbed[1] = 1
peturbed

10-element BitVector:
 1
 1
 1
 0
 1
 0
 0
 0
 0
 0

In [14]:
# Recall the 5th memory
recall = memories[:,5]
peturbed = recall

# Perturb this memory by flipping a random bit
peturbed = peturb_memory!(peturbed,rand(1:length(peturbed)))
peturbed = peturb_memory!(peturbed,rand(1:length(peturbed)))
peturbed = peturb_memory!(peturbed,rand(1:length(peturbed)))
peturbed = peturb_memory!(peturbed,rand(1:length(peturbed)))
peturbed = peturb_memory!(peturbed,rand(1:length(peturbed)))
peturbed = peturb_memory!(peturbed,rand(1:length(peturbed)))
peturbed = peturb_memory!(peturbed,rand(1:length(peturbed)))
peturbed = peturb_memory!(peturbed,rand(1:length(peturbed)))
peturbed = peturb_memory!(peturbed,rand(1:length(peturbed)))
peturbed = peturb_memory!(peturbed,rand(1:length(peturbed)))

print(peturbed)
print('\n',recall)
# Perform learning
steps = 100

# Find the initial distance between the network and the recall
distance = norm(peturbed-recall)
print(distance)

# for ii in 1:steps
#     # Take a step with input of the peturbed memory
#     update = learning_step(weights, peturbed, activations)
#     # Calculate the distance between update and recall
#     distance = norm(update-recall)
#     # Print the distance
#     print('\n', distance)
#     # Assign recall to update 
#     peturbed = update
# end

Bool[0, 1, 1, 0, 1, 0, 0, 0, 0, 0]
Bool[0, 1, 1, 0, 1, 0, 0, 0, 0, 0]0.0

# Memories vs errors analysis

In [37]:
# Set the number of neurons 
neurons = 10

# Set an array of different memory sizes to store from 1 to 100
memories_array = 2:100

# Initialse an array the length of memories to store the errors 
errors = zeros(Int8,(length(memories_array),1))

# Iterate through the different memory sizes 
for ii in 1:length(memories_array)
    # Generate random memories
    memories = generate_memories(neurons, memories_array[ii])
    # Initialise weights
    weights = initialise_weights(memories)
    # Perform learning
    steps = 100
    # Activations as array of length neurons all set to 0 
    activations = zeros(Int8,(neurons,1))
    # Select a random memory to recall
    recall = memories[:,rand(1:length(memories[1,:]))]
    update = recall
    
    # Perform learning
    for jj in 1:steps
        # Perform learning
        update = learning_step(weights,update,activations)
        # Calculate the distance between update and recall
        distance = norm(update-recall)
        # Print the distance
        print('\n', distance)
    end
    # Store the error in the array
    errors[ii] = distance
end


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

In [69]:
# Try for 100 memories in a 10 neuron network
neurons = 10 
memories_size = 100

# Geenerate the memories
memories = generate_memories(neurons,memories_size)

# Set the weights 
weights = initialise_weights(memories)

# Select a random memory to recall 
recall = memories[:,rand(1:length(memories[1,:]))]

10-element BitVector:
 1
 1
 1
 0
 1
 1
 1
 0
 1
 1

In [70]:
# Peturb two elements
peturbed = copy(recall)
peturbed = peturb_memory(peturbed,rand(1:length(peturbed)))
peturbed = peturb_memory(peturbed,rand(1:length(peturbed)))

10-element BitVector:
 1
 1
 1
 0
 1
 0
 0
 0
 1
 1

In [71]:
recall

10-element BitVector:
 1
 1
 1
 0
 1
 1
 1
 0
 1
 1

In [73]:
# Distance between the peturbed memory and the recall
distance = jaccard(peturbed,recall)

0.25

In [45]:
# Perform learning and see if network can stabilise on this memory
steps = 100
activations = zeros(Int8,(neurons,1))
for ii in 1:steps
    distance = norm(peturbed-recall)
    print('\n', distance)
    update = learning_step(weights,peturbed,activations)
    peturbed = update
end


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0